In [9]:
import requests
import pandas as pd
import numpy as np
import datetime

def etherscan_request(action, 
                      address='0x7bea39867e4169dbe237d55c8242a8f2fcdcc387', 
                      api_key='Y8YP3J3BTG1DRZZPQ3F2I41E88I9GSUXNU', 
                      startblock=0, 
                      endblock=99999999, 
                      sort='desc'):
    
    base_url = 'https://api.etherscan.io/api'
    params = {
        'module': 'account',
        'action': action,
        'address': address,
        'startblock': startblock,
        'endblock': endblock,
        'sort': sort,
        'apikey': api_key
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    
    data = response.json()
    
    if data['status'] != '1':
        raise Exception(f"API returned an error: {data['result']}")
    
    df = pd.DataFrame(data['result'])

    expected_columns = ['hash', 'blockNumber', 'timeStamp', 'from', 'to', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed', 'confirmations', 'tokenSymbol', 'value', 'tokenName']
    
    for col in expected_columns:
        if col not in df.columns:
            raise Exception(f"Expected column '{col}' is missing from the response")
    
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    # Set Transaction Value in Appropriate Format
    df['og_value'] = df['value'].copy()
    df['value'] = np.where(df['tokenDecimal']=='6', df['value']/100000, df['value']/1000000000000000000)

    # Sort by timestamp in descending order and select the most recent 10,000 trades
    df['timeStamp'] = pd.to_numeric(df['timeStamp'])
    df = df.sort_values(by='timeStamp', ascending=False).head(10000)
    
    consolidated_data = {}

    for index, row in df.iterrows():
        tx_hash = row['hash']
        
        if tx_hash not in consolidated_data:
            consolidated_data[tx_hash] = {
                'blockNumber': row['blockNumber'],
                'timeStamp': row['timeStamp'],
                'hash': tx_hash,
                'from': row['from'],
                'to': row['to'],
                'WETH_value': 0,
                'USDC_value': 0,
                'tokenName_WETH': '',
                'tokenName_USDC': '',
                'gas': row['gas'],
                'gasPrice': row['gasPrice'],
                'gasUsed': row['gasUsed'],
                'cumulativeGasUsed': row['cumulativeGasUsed'],
                'confirmations': row['confirmations']
            }
        
        if row['tokenSymbol'] == 'WETH':
            consolidated_data[tx_hash]['WETH_value'] = row['value']
            consolidated_data[tx_hash]['tokenName_WETH'] = row['tokenName']
        elif row['tokenSymbol'] == 'USDC':
            consolidated_data[tx_hash]['USDC_value'] = row['value']
            consolidated_data[tx_hash]['tokenName_USDC'] = row['tokenName']

    return pd.DataFrame.from_dict(consolidated_data, orient='index'), df

consolidated, transactions = etherscan_request('tokentx')
consolidated['time'] = consolidated['timeStamp'].apply(lambda x:datetime.datetime.fromtimestamp(x))
consolidated


AttributeError: type object 'datetime.datetime' has no attribute 'fromstimestamp'

In [13]:
import datetime


,blockNumber,timeStamp,hash,from,to,WETH_value,USDC_value,tokenName_WETH,tokenName_USDC,gas,gasPrice,gasUsed,cumulativeGasUsed,confirmations,time
0xc6f8f12077d5c67a69833fe6446fc529af872c1e964c9621416fb708bd58d255,20342713,1721417843,0xc6f8f12077d5c67a69833fe6446fc529af872c1e964c...,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.564981,20000.00000,Wrapped Ether,USDC,640420,7670839168,306807,9195319,8020,2024-07-19 12:37:23
0x682c75dfd8ac8d4af830865053dd3f3eec565da9447176cd06bd27493da8a83c,20342683,1721417483,0x682c75dfd8ac8d4af830865053dd3f3eec565da94471...,0xad3b67bca8935cb510c8d18bd45f0b94f54a968f,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.256916,9085.13897,Wrapped Ether,USDC,559962,7228306132,281399,10245699,8050,2024-07-19 12:31:23
0x449b0ef9089aa15b2ea41101e070ec501c0800099c0be0d5cfe34e91660c6286,20342628,1721416823,0x449b0ef9089aa15b2ea41101e070ec501c0800099c0b...,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.776401,27419.27641,Wrapped Ether,USDC,244654,4568019148,122351,12201783,8105,2024-07-19 12:20:23
0x87bfdc19631ced6b501aac852d6f2d71e310ed15680b69f38be3945101f8cf7a,20342578,1721416223,0x87bfdc19631ced6b501aac852d6f2d71e310ed15680b...,0x4a14347083b80e5216ca31350a2d21702ac3650d,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.307368,10840.00000,Wrapped Ether,USDC,625000,8976799529,353733,1204495,8155,2024-07-19 12:10:23
0xa5bca4df8981e8c73ea539c53064ae9ee0dc3bc77f6980a29e5bda651a64c500,20342557,1721415971,0xa5bca4df8981e8c73ea539c53064ae9ee0dc3bc77f69...,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,1.774761,62425.48550,Wrapped Ether,USDC,244830,6205397986,122408,2289629,8176,2024-07-19 12:06:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0x3f70effeb70f575d7fa0a41022e5ac5ce782303a3c71ed71c27bb1b34a7be93d,19237122,1708046027,0x3f70effeb70f575d7fa0a41022e5ac5ce782303a3c71...,0x74de5d4fcbf63e00296fd95d33236b9794016631,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,1.196652,34150.88897,Wrapped Ether,USDC,254742,24997459381,204954,21709019,1113611,2024-02-15 17:13:47
0x4b1aeb564fb971fed63f8387dbd2b8f1950e6cde11dc14b68bccf315d8c8de1c,19237118,1708045979,0x4b1aeb564fb971fed63f8387dbd2b8f1950e6cde11dc...,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,3.047907,86855.82285,Wrapped Ether,USDC,244672,26075719253,122268,29487646,1113615,2024-02-15 17:12:59
0xe8194412c4a90d72f7d4ec5bc6d8db4cd15c39b2a84b02eba7f54cfff5cda7c2,19237084,1708045571,0xe8194412c4a90d72f7d4ec5bc6d8db4cd15c39b2a84b...,0xb6569481dccddd527c2b0e8ba32f494e52224ca1,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.401821,11437.01066,Wrapped Ether,USDC,689219,22317230186,571479,1371314,1113649,2024-02-15 17:06:11
0xe0bff55ac94ba32a6f343a7db077bb52b9dc01b043adfd0c29704e8e851d00ff,19236165,1708034351,0xe0bff55ac94ba32a6f343a7db077bb52b9dc01b043ad...,0xf3de3c0d654fda23dad170f0f320a92172509127,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.931798,25998.61534,Wrapped Ether,USDC,284253,33982441371,216027,6275388,1114568,2024-02-15 13:59:11
